In [1]:
import os
import pandas as pd
import numpy as np
import urllib
from datetime import datetime,timedelta

In [2]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data'
os.chdir(path)

today = datetime.now().strftime("%Y-%m-%d")
yesterday = datetime.strftime(datetime.now() - timedelta(1), "%Y-%m-%d")

csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data/csv.csv'
f = open(csv,'w+')

fm_txt = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data\five_minute\missing_data_report_' + str(yesterday) + '.txt'
hr_txt = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data\hourly\missing_data_report_' + str(yesterday) + '.txt'
dy_txt = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\missing_data\daily\missing_data_report_' + str(yesterday) + '.txt'

g = open(fm_txt,'a')
h = open(hr_txt,'a')
i = open(dy_txt,'a')

xl_file = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\meta_files\colocation_metadata.xlsx'

In [3]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3], axis=1)

In [12]:
n=0
for index,rows in meta.iterrows():
    try:
        print('Pulling 5-minute data from ' + str(meta.ID[n]) + ' ....')
        urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=five_minute&sids=' 
                                   + str(meta.ID[n])) + '&sdate=' + str(yesterday) + '&edate=' + str(today) + 
                                   '&elems=tmean,rh,vp,sr,ws,wind_vec,pp,st5,st15,gust,gusttm,gustdir', filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
        df.columns = ['station','date','tmean','rh','vp','sr','ws','wind_vec','pp','st5','st15','gust','gusttm','gustdir']
        df.drop(df.tail(1).index,inplace=True)
    except Exception as e:
        print('Something went wrong while pulling the 5-minute data from ' + str(meta.ID[n]) + ': ' + e.args[0])    
    try:
        print('Repopulating the 5-minute data from ' + str(meta.ID[n]) + ' to find missing timesteps....')
        df['date'] = pd.to_datetime(df.date)
        r = pd.date_range(start=yesterday,end=today,freq='5min')
        df = df.set_index('date').reindex(r,copy=False).rename_axis('date')
    except Exception as e:
        print('Something went wrong while repopulating the 5-minute data from ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Checking the 5-minute data from ' + str(meta.ID[n]) + ' for missing values....')
        d = 0
        for index,rows in df.iterrows():
            if df.tmean[d] == np.nan:
                temp_string = (str(df.station[d]) + ' reported missing 5-minute temperature data at ' + str(df.date[d]) + '\n')
                g.write(temp_string)
            elif df.rh[d] == np.nan:
                rh_string = (str(df.station[d]) + ' reported missing relative humidity data at ' + str(df.date[d]) + '\n')
                g.write(rh_string)
            elif df.vp[d] == np.nan:
                vp_string = (str(df.station[d]) + ' reported missing 5-minute vapor pressure data at ' + str(df.date[d]) + '\n')
                g.write(vp_string)
            elif df.sr[d] == np.nan:
                sr_string(str(df.station[d]) + ' reported missing 5-minute solar radiation data at ' + str(df.date[d]) + '\n')
            elif df.ws[d] == np.nan:
                ws_string = (str(df.station[d]) + ' reported missing 5-minute wind speed data at ' + str(df.date[d]) + '\n')
                g.write(ws_string)
            elif df.wind_vec == np.nan:
                wvec_string = (str(df.station[d]) + ' reported missing 5-minute wind direction data at ' + str(df.date[d]) + '\n')
                g.write(wvec_string)
            elif df.pp == np.nan:
                pp_string = (str(df.station[d]) + ' reported missing 5-minute precipitation data at ' + str(df.date[d]) + '\n')
                g.write(pp_string)
            elif df.st5 == np.nan:
                stfive_string = (str(df.station[d]) + ' reported missing 5-minute 5cm soil temp at ' + str(df.date[d]) + '\n')
                g.write(stfive_string)
            elif df.st15 == np.nan:
                stfifteen_string = (str(df.station[d]) + ' reported missing 5-minute 15cm soil temp at ' + str(df.date[d]) + '\n')
                g.write(stfifteen_string)
            elif df.gust == np.nan:
                gust_string = (str(df.station[d]) + ' reported missing 5-minute gust data ' + str(df.date[d]) + '\n')
                g.write(gust_string)
            elif df.gusttm == np.nan:
                gusttm_string = (str(df.station[d]) + ' reported missing 5-minute gust time data at ' + str(df.date[d]) + '\n')
                g.write(gusttm_string)
            elif df.gustdir == np.nan:
                gustdir_string = (str(df.station[d]) + ' reported missing 5-minute gust direction data at ' + str(df.date[d]) + '\n')
                g.write(gustdir_string)
            d = d+1
    except Exception as e:
        print('Something went wrong while checking the 5-minute data from ' + str(meta.ID[n]) + 'for missin values: ' + e.args[0])
    try:
        print('Pulling hourly data from ' + str(meta.ID[n]) + ' to check for missing values....')
        urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=hourly&sids='
                                   + str(meta.ID[n]) + '&sdate=' + str(yesterday) + '&edate=' + str(today) +
                                   '&elems=tmean,rh,vp,sr,ws,wind_vec,wind_std,pp,st5,st15,gust,gusttm,gustdir', filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
        df.columns = ['station','date','tmean','rh','vp','sr','ws','wind_vec','wind_std','pp','st5','st15','gust','gusttm','gustdir']
        df.drop(df.tail(1).index,inplace=True)
    except Exception as e:
        print('Something went wrong while pulling the hourly data from ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Repopulating the hourly data from ' + str(meta.ID[n]) + ' to to check for missing timesteps....')
        df['date'] = pd.to_datetime(df.date)
        r = pd.date_range(start=yesterday,end=today,freq='H')
        df = df.set_index('date').reindex(r,copy=False).rename_axis('date')
    except Exception as e:
        print('Something went wrong while repopulating the hourly data from ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Checking hourly data from ' + str(meta.ID[n]) + ' for missing values')
        a = 0
        for index,rows in df.iterrows():
            if df.tmean[d] == np.nan:
                temp_string = (str(df.station[a]) + ' reported missing hourly temperature data at ' + str(df.date[a]) + '\n')
                h.write(temp_string)
            elif df.rh[d] == np.nan:
                rh_string = (str(df.station[a]) + ' reported missing hourly relative humidity data at ' + str(df.date[a]) + '\n')
                h.write(rh_string)
            elif df.vp[d] == np.nan:
                vp_string = (str(df.station[a]) + ' reported missing hourly vapor pressure data at ' + str(df.date[a]) + '\n')
                h.write(vp_string)
            elif df.sr[d] == np.nan:
                sr_string(str(df.station[a]) + ' reported missing hourly solar radiation data at ' + str(df.date[a]) + '\n')
            elif df.ws[d] == np.nan:
                ws_string = (str(df.station[a]) + ' reported missing hourly wind speed data at ' + str(df.date[a]) + '\n')
                h.write(ws_string)
            elif df.wind_vec == np.nan:
                wvec_string = (str(df.station[a] + ' reported missing hourly wind direction data at ' + str(df.date[a]) + '\n')
                h.write(wvec_string)
            elif df.pp == np.nan:
                pp_string = (str(df.station[a]) + ' reported missing hourly precipitation data at ' + str(df.date[a]) + '\n')
                h.write(pp_string)
            elif df.st5 == np.nan:
                stfive_string = (str(df.station[a]) + ' reported missing hourly 5cm soil temp at ' + str(df.date[a]) + '\n')
                h.write(stfive_string)
            elif df.st15 == np.nan:
                stfifteen_string = (str(df.station[a] + ' reported missing hourly 15cm soil temp at ' + str(df.date[a]) + '\n')
                h.write(stfifteen_string)
            elif df.gust == np.nan:
                gust_string = (str(df.station[a]) + ' reported missing hourly gust data ' + str(df.date[a]) + '\n')
                h.write(gust_string)
            elif df.gusttm == np.nan:
                gusttm_string = (str(df.station[a]) + ' reported missing hourly gust time data at ' + str(df.date[a]) + '\n')
                h.write(gusttm_string)
            elif df.gustdir == np.nan:
                gustdir_string = (str(df.station[a]) + ' reported missing hourly gust direction data at ' + str(df.date[a]) + '\n')
                h.write(gustdir_string)
            a = a+1
    except Exception as e:
        print('Something went wrong while checking the hourly data from ' + str(meta.ID[n]) + ' for missing values: ' + e.args[0])
    try:
        print('Pulling daily data from ' + str(meta.ID[n]) + 'to check for missing values...')
        urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate='
                                   + str(yesterday) + '&edate=' + str(today) + 
                                   '&elems=tave,tmax,mxtm,tmin,mntm,vp,rhmax,rhmxtm,rhmin,rhmntm,sr,wrun,pp,st5mx,st5mxtm,st5mn,st5mntm,st15mx,st15mxtm,st15mn,st15mntm,volts,gust,gusttm,gustdir',
                                   filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
        df.columns = ['station','date','tave','tmax','mxtm','tmin','mntm','vp','rhmax','rhmxtm','rhmin','rhmntm','sr','wrun','pp','st5mx','st5mxtm','st5mn','st5mntm','st15mx','st15mxtm','volts','gust','gusttm','gustdir']
        df.drop(df.tail(1).index,inplace=True)
    except Exception as e:
        print('Something went wrong while pulling the daily data from ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Repopulating the daily data from ' + str(meta.ID[n]) + ' to check for missing values...')
        df['date'] = pd.to_datetime(df.date)
        r = pd.date_range(start=yesterday,end=today,freq='D')
        df = df.set_index('date').reindex(r,copy=False).rename_axis('date')
    except Exception as e:
        print('Something went wrong while repopulating the daily data from ' + str(meta.ID[n]) + ' to check for missing values: ' + e.args[0])
    try:
        print('Checking the daily data from ' + str(meta.ID[n]) + ' for missing values')
        b = 0
        for index, rows in df.iterrows():
            if df.tave[b] == np.nan:
                tave_string = (str(df.station[b]) + ' reported missing daily average temperature on ' + str(df.date[b]) + '\n')
                i.write(tave_string)
            elif df.tmax[b] == np.nan:
                tmax_string = (str(df.station[b]) + ' reported missing daily maximum temperature on ' + str(df.date[b]) + '\n')
                i.write(tmax_string)
            elif df.mxtm[b] == np.nan:
                mxtm_string = (str(df.station[b]) + ' reported missing daily max temp time on ' + str(df.date[b]) + '\n')
                i.write(mxtm_string)
            elif df.tmin[b] == np.nan:
                tmin_string = (str(df.station[b]) + ' reported missing daily minimum temperature on ' + str(df.date[b]) + '\n')
                i.write(tmin_string)
            elif df.mntm[b] == np.nan:
                mntm_string = (str(df.station[b]) + ' reported missing daily min temp time on ' + str(df.date[b]) + '\n')
                i.write(mntm_string)
            elif df.vp[b] == np.nan:
                vp_string = (str(df.station[b]) + ' reported missing daily vapor pressure on ' + str(df.date[b]) + '\n')
                i.write(vp_string)
            elif df.rhmax[b] == np.nan:
                rhmax_string = (str(df.station[b]) + ' reported missing daily max rh on ' + str(df.date[b]) + '\n')
                i.write(rhmax_string)
            elif df.rhmxtm[b] == np.nan:
                rhmxtm_string = ( str(df.station[b]) + ' rerported missing daily max rh time on ' + str(df.date[b]) + '\n')
                i.write(rhmxtm_string)
            elif df.rhmin[b] == np.nan:
                rhmin_string = (str(df.station[b]) + ' reported missing min rh on ' + str(df.date[b]) + '\n')
                i.write(rhmin_string)
            elif df.rhmntm[b] == np.nan:
                rhmntm_string = (str(df.station[b]) + ' reported missing mn rh time on ' + str(df.date[b]) + '\n')
                i.write(rhmntm_string)
            elif df.sr[b] == np.nan:
                sr_string = (str(df.station[b]) + ' reported missing daily sr on ' + str(df.date[b]) + '\n')
                i.write(sr_string)
            elif df.wrun == np.nan:
                wrun_string = (str(df.station[b]) + ' reported missing wind run on ' + str(df.date[b]) + '\n')
                i.write(wrun_string)
            elif df.pp == np.nan:
                pp_string = (str(df.station[b]) + ' reported missing daily precipitation on ' + str(df.date[b]) + '\n')
                                    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 101)